# 📈 Professional Stock Market Prediction with LSTM

This notebook provides a complete, production-ready guide to implementing a sophisticated stock prediction model using Long Short-Term Memory (LSTM) networks. The methodology is synthesized from common findings in financial machine learning research, focusing on high-performing architectures and techniques to create a practical and robust tool.

**Core Methodology:** The implementation is based on a common and effective LSTM architecture. This model uses a 60-day time step (window) and incorporates On-Balance Volume (OBV) as a key feature alongside the closing price. Such architectures have been shown in various studies to be effective for time-series forecasting.

### Key Features of this Notebook:
* **Interactive & Customizable:** An interactive widget allows you to easily change the stock ticker and other key parameters.
* **End-to-End Pipeline:** Covers the entire workflow from data acquisition and feature engineering to model training, evaluation, and visualization.
* **Automated Evaluation:** Automatically evaluates the LSTM model against a naive baseline to provide a clear performance benchmark.
* **Robust & Reliable:** Includes input validation, error handling, and the saving of both the model and the necessary data scalers for reliable future predictions.
* **Practical Outputs:** Provides functionality to save the trained model, load it for future use, and export predictions to a CSV file.

### Theoretical Backing (Adaptive Markets Hypothesis)
While the traditional Efficient Market Hypothesis (EMH) suggests that market prices are unpredictable, this notebook's approach is better supported by the **Adaptive Markets Hypothesis (AMH)**. The AMH posits that markets are not perfectly efficient and that temporary inefficiencies arise, which advanced machine learning models like LSTM can learn to exploit for prediction.

## 📚 Table of Contents

1.  [Setup & Dependencies](#section1)
2.  [Configuration & Parameters](#section2)
3.  [Data Acquisition & Exploration](#section3)
4.  [Feature Engineering (Technical Indicators)](#section4)
5.  [Data Preprocessing](#section5)
6.  [LSTM Model Implementation](#section6)
7.  [Model Training](#section7)
8.  [Prediction & Performance Evaluation](#section8)
    * [8.1 Baseline Model Comparison](#subsection8_1)
    * [8.2 Visualizing Results](#subsection8_2)
    * [8.3 Exporting Predictions](#subsection8_3)
9.  [Saving and Loading the Model & Scalers](#section9)
10. [Live Prediction on New Data](#section10)
11. [Advanced Topics & Future Work](#section11)

## <a id="section1"></a>
## 1. Setup & Dependencies

### What's happening here?
We are installing and importing all the necessary Python libraries for our project.
- `yfinance`: To fetch historical stock market data from Yahoo Finance.
- `pandas` & `numpy`: For data manipulation and numerical operations.
- `scikit-learn`: For data preprocessing (scaling with `MinMaxScaler`, saving scalers with `joblib`) and evaluation metrics.
- `tensorflow`: The deep learning framework to build and train our LSTM model.
- `plotly`: To create interactive and professional-looking visualizations.

The code block also checks for GPU availability to accelerate model training, a key performance optimization technique.

**Run this cell to install and import dependencies.**

In [ ]:
# Install libraries
!pip install yfinance pandas numpy scikit-learn tensorflow plotly joblib kaleido -q

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import gc
from datetime import datetime
import joblib # For saving and loading scalers

# The following imports are for Google Colab. If running elsewhere, you might need to adapt this.
try:
    from google.colab import drive, files
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# Suppress TensorFlow warnings
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [ ]:
# --- GPU Check ---
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('⚠️ GPU device not found. Training will be on CPU.')
else:
  print(f'✅ Found GPU at: {device_name}. Training will be GPU-accelerated.')

# --- Reproducibility ---
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds()
print("✅ Dependencies installed and imported successfully.")

## <a id="section2"></a>
## 2. Configuration & Parameters

### What's happening here?
This interactive form centralizes all key parameters, making the notebook highly reusable and easy to customize. The default values are based on configurations commonly found to perform well in financial forecasting research.

- **Model:** A stacked LSTM architecture using a 60-Day Time Step with OBV.
- **Stock:** Enter any valid Yahoo Finance ticker symbol. `AAPL` is the default.
- **Date Range:** A long period provides more data for robust training. The end date dynamically defaults to the current day.
- **Window Size:** Set to 60, a common choice that has been found to be empirically effective in multiple studies.
- **Features:** The list includes `Close` and `OBV`. Research suggests that combining price with volume-based indicators can improve model performance.
- **Hyperparameters:** These values (units, dropout, learning rate) are example settings that serve as a strong starting point for tuning.

In [ ]:
# --- General Configuration ---
STOCK_TICKER = "AAPL"  # @param {type:"string"}
START_DATE = "2010-01-01"  # @param {type:"date"}
END_DATE = datetime.now().strftime('%Y-%m-%d') # @param {type:"date"}
TRAIN_TEST_SPLIT_RATIO = 0.85  # @param {type:"slider", min:0.7, max:0.9, step:0.05}

# --- Model & Feature Configuration ---
WINDOW_SIZE = 60  # @param {type:"integer"}
# The model will use 'Close' and 'OBV'. You can experiment by adding more features here.
# e.g., ['Close', 'OBV', 'RSI', 'MACD']
FEATURES_TO_USE = ['Close', 'OBV']

# --- LSTM Hyperparameters (example values) ---
LSTM_UNITS = 104  # @param {type:"integer"}
DROPOUT_RATE = 0.266  # @param {type:"number"}
LEARNING_RATE = 0.00369  # @param {type:"number"}
LOSS_FUNCTION = 'mean_squared_error'
OPTIMIZER = Adam(learning_rate=LEARNING_RATE)

# --- Training Configuration ---
EPOCHS = 100  # @param {type:"integer"}
BATCH_SIZE = 32  # @param {type:"integer"}
EARLY_STOPPING_PATIENCE = 10

# --- File Paths for Saving Artifacts ---
# We will save the model, and the scalers, to be able to make predictions later.
SAVE_DIR = "model_artifacts"
os.makedirs(SAVE_DIR, exist_ok=True)
MODEL_PATH = os.path.join(SAVE_DIR, f'{STOCK_TICKER}_best_model.h5')
FEATURE_SCALER_PATH = os.path.join(SAVE_DIR, f'{STOCK_TICKER}_feature_scaler.joblib')
TARGET_SCALER_PATH = os.path.join(SAVE_DIR, f'{STOCK_TICKER}_target_scaler.joblib')

print("✅ Configuration parameters set.")
print(f"Target Stock: {STOCK_TICKER}")
print(f"Features for Model: {FEATURES_TO_USE}")
print(f"Window Size: {WINDOW_SIZE} days")

## <a id="section3"></a>
## 3. Data Acquisition & Exploration

### What's happening here?
We use the `yfinance` library to download historical stock data. This section includes robust error handling to ensure a valid ticker is provided and data is successfully fetched.

1.  **Fetch Data:** `yf.download()` pulls the historical market data.
2.  **Input Validation:** A `try-except` block validates the ticker symbol. If no data is returned, it prints a clear error message and stops execution.
3.  **Inspect Data:** We display the first few rows and check for missing values, which is a critical preprocessing step.
4.  **Visualize:** An interactive candlestick chart helps in understanding the data's trend and volatility.

In [ ]:
# --- Fetch Data with Error Handling ---
data = pd.DataFrame()
try:
    data = yf.download(STOCK_TICKER, start=START_DATE, end=END_DATE, progress=False)
    if data.empty:
        raise ValueError(f"No data found for ticker '{STOCK_TICKER}'. It may be an invalid symbol or delisted.")
    print(f"✅ Successfully downloaded {len(data)} data points for {STOCK_TICKER}.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")

In [ ]:
# --- Proceed only if data was downloaded successfully ---
if not data.empty:
    # --- Inspect Data ---
    print("\n--- First 5 Rows of Data ---")
    display(data.head())

    print("\n--- Data Info & Missing Values ---")
    # Handling missing values is a critical preprocessing step.
    if data.isnull().sum().any():
        print("Missing values found. Applying forward-fill.")
        data.ffill(inplace=True)
        data.dropna(inplace=True) # Drop any remaining NaNs at the beginning
    else:
        print("No missing values found.")
else:
    print("\nStopping execution due to data fetching failure.")

In [ ]:
if not data.empty:
    # --- Visualize Historical Data ---
    fig_explore = go.Figure(data=[go.Candlestick(x=data.index,
                                               open=data['Open'],
                                               high=data['High'],
                                               low=data['Low'],
                                               close=data['Close'],
                                               name='Candlestick')])

    fig_explore.update_layout(
        title=f'Historical Price Data for {STOCK_TICKER}',
        xaxis_title='Date',
        yaxis_title='Stock Price (USD)',
        xaxis_rangeslider_visible=False,
        template='plotly_dark'
    )
    fig_explore.show()

## <a id="section4"></a>
## 4. Feature Engineering (Technical Indicators)

### What's happening here?
We derive new features (technical indicators) from the raw price and volume data. Research highlights that these indicators can provide the model with crucial insights into market dynamics, momentum, and volatility, thereby enhancing predictive accuracy.

We implement functions for each indicator, with their formulas included as comments. The critical feature for our model is **On-Balance Volume (OBV)**.

In [ ]:
if not data.empty:
    # --- Indicator Implementation ---
    def calculate_sma(data, window):
        # Formula: SMA = (P1 + P2 + ... + Pn) / n
        return data['Close'].rolling(window=window).mean()

    def calculate_rsi(data, window=14):
        # Formula: RSI = 100 - (100 / (1 + RS)) where RS = Avg Gain / Avg Loss
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        # Add a small epsilon to avoid division by zero
        rs = gain / (loss + 1e-8)
        return 100 - (100 / (1 + rs))

    def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
        # Formula: MACD = EMA_short - EMA_long
        short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
        long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
        macd = short_ema - long_ema
        signal_line = macd.ewm(span=signal_window, adjust=False).mean()
        return macd, signal_line

    def calculate_obv(data):
        # Formula: If Close > Prev_Close, OBV = Prev_OBV + Volume
        # If Close < Prev_Close, OBV = Prev_OBV - Volume
        obv = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()
        return obv

In [ ]:
if not data.empty:
    # --- Apply Indicators to DataFrame ---
    data['SMA_15'] = calculate_sma(data, 15)
    data['SMA_45'] = calculate_sma(data, 45)
    data['RSI'] = calculate_rsi(data)
    data['MACD'], data['Signal_Line'] = calculate_macd(data)
    data['OBV'] = calculate_obv(data)

    # Drop initial NaN values created by rolling windows
    data.dropna(inplace=True)

    print("✅ Technical indicators calculated and added to the dataframe.")
    print("--- Data with New Features ---")
    display(data.head())

## <a id="section5"></a>
## 5. Data Preprocessing

### What's happening here?
This is a critical phase to prepare the data for our LSTM model.

1.  **Feature Selection:** We select the subset of features specified in our configuration (`Close` and `OBV` by default).
2.  **Normalization:** We use `MinMaxScaler` to scale all selected features to a range of [0, 1]. This is vital for LSTM models as it helps with efficient optimization. We use two separate scalers: one for the input features and one for the target variable (`Close`). This is a best practice that prevents data leakage from other features into the target's scaling, making the inverse transform of our prediction more robust.
3.  **Sequence Generation (Sliding Window):** LSTMs require data in a sequential format. A "sliding window" of `WINDOW_SIZE` (60) days of feature data is used as input (`X`) to predict the closing price of the next day (`y`).
4.  **Data Splitting:** The data is split chronologically into training and testing sets to prevent data leakage.

In [ ]:
if not data.empty:
    # --- 1. Feature Selection ---
    features = data[FEATURES_TO_USE]
    print(f"Selected features for model: {features.columns.to_list()}")

    # --- 2. Normalization ---
    # We use two separate scalers. This is best practice.
    # feature_scaler is for all input features (e.g., Close, OBV).
    # target_scaler is only for the target ('Close'), making inverse transform straightforward.
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    target_scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_features = feature_scaler.fit_transform(features)
    scaled_target = target_scaler.fit_transform(data[['Close']])

    print("\n--- Data after Min-Max Scaling ---")
    print("Shape of scaled data:", scaled_features.shape)

In [ ]:
if not data.empty:
    # --- 3. Sequence Generation ---
    def create_sequences(data_features, data_target, window_size):
        X, y = [], []
        for i in range(window_size, len(data_features)):
            X.append(data_features[i-window_size:i, :])
            y.append(data_target[i, 0]) # Target is the scaled 'Close' price
        return np.array(X), np.array(y)

    X, y = create_sequences(scaled_features, scaled_target, WINDOW_SIZE)

    print(f"\n--- Sequences Created (Window Size: {WINDOW_SIZE}) ---")
    print(f"Shape of X (input sequences): {X.shape}")
    print(f"Shape of y (target values): {y.shape}")

In [ ]:
if not data.empty:
    # --- 4. Data Splitting ---
    split_index = int(len(X) * TRAIN_TEST_SPLIT_RATIO)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    print(f"\n--- Data Split (Train: {TRAIN_TEST_SPLIT_RATIO*100:.0f}%, Test: {(1-TRAIN_TEST_SPLIT_RATIO)*100:.0f}%) ---")
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
    print("✅ Data preprocessing complete.")

    # --- Memory Management ---
    del features, scaled_features, scaled_target
    gc.collect()
    print("\n🧹 Unused dataframes cleared from memory.")

## <a id="section6"></a>
## 6. LSTM Model Implementation

### What's happening here?
We define the architecture of our LSTM model using TensorFlow's Keras API. The structure is a stacked LSTM, a common approach for learning complex temporal patterns.

1.  **`Sequential` Model:** A linear stack of layers.
2.  **First `LSTM` Layer:** Receives the input sequences. `return_sequences=True` is crucial for stacking another LSTM layer on top.
3.  **Second `LSTM` Layer:** Learns higher-level temporal representations.
4.  **`Dropout` Layer:** A powerful technique to prevent overfitting by randomly setting a fraction of input units to 0 during training.
5.  **`Dense` Output Layer:** A single neuron to output the predicted (normalized) stock price.
6.  **`compile()`:** We configure the model for training with the Adam optimizer and Mean Squared Error loss function.

In [ ]:
if not data.empty:
    # --- Define the Model Architecture ---
    model = Sequential()

    # First LSTM Layer with Dropout
    model.add(LSTM(units=LSTM_UNITS,
                   return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(DROPOUT_RATE))

    # Second LSTM Layer
    model.add(LSTM(units=LSTM_UNITS, return_sequences=False))
    model.add(Dropout(DROPOUT_RATE))

    # Dense Output Layer
    model.add(Dense(units=1))

    # --- Compile the Model ---
    model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNCTION)

    print("✅ LSTM Model built successfully.")
    model.summary()

## <a id="section7"></a>
## 7. Model Training

### What's happening here?
Now we train our compiled LSTM model on the preprocessed training data.

1.  **Callbacks:** We define two helpful callbacks:
    - **`EarlyStopping`:** This monitors the validation loss and stops the training process if it doesn't improve for a set number of epochs (`patience`). This is a critical best practice to prevent overfitting and save computation time.
    - **`ModelCheckpoint`:** This saves the best version of the model to a file (`MODEL_PATH`) during training.
2.  **`model.fit()`:** This is the main training function. We pass the training data, epochs, batch size, and a validation split to monitor performance on unseen data during training.
3.  **Visualize Training History:** After training, we plot the training vs. validation loss. This plot is essential for diagnosing issues like overfitting or underfitting.

In [ ]:
if not data.empty:
    # --- Define Callbacks ---
    early_stop = EarlyStopping(monitor='val_loss',
                               patience=EARLY_STOPPING_PATIENCE,
                               restore_best_weights=True,
                               verbose=1)

    checkpoint = ModelCheckpoint(filepath=MODEL_PATH,
                                 monitor='val_loss',
                                 save_best_only=True,
                                 verbose=1)
    
    # --- Train the Model ---
    print("\n--- Starting Model Training ---")
    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1, # Use 10% of training data for validation
        callbacks=[early_stop, checkpoint],
        verbose=1
    )
    print("\n--- Model Training Complete ---")
    
    # --- Save the scalers ---
    # It's crucial to save the scalers used to preprocess the data
    joblib.dump(feature_scaler, FEATURE_SCALER_PATH)
    joblib.dump(target_scaler, TARGET_SCALER_PATH)
    print(f"✅ Model and scalers saved to '{SAVE_DIR}/' directory.")

In [ ]:
if not data.empty and 'history' in locals():
    # --- Visualize Training History ---
    fig_history = go.Figure()
    fig_history.add_trace(go.Scatter(y=history.history['loss'], name='Training Loss'))
    fig_history.add_trace(go.Scatter(y=history.history['val_loss'], name='Validation Loss'))
    fig_history.update_layout(title='Model Training and Validation Loss Over Epochs',
                              xaxis_title='Epochs',
                              yaxis_title='Loss (MSE)',
                              template='plotly_dark')
    fig_history.show()

## <a id="section8"></a>
## 8. Prediction & Performance Evaluation

### What's happening here?
With our model trained, we evaluate its performance on the unseen test dataset to understand how well it generalizes.

1.  **Load Best Model:** We load the best model saved by `ModelCheckpoint` to ensure we evaluate the top-performing iteration.
2.  **Make Predictions:** We use `model.predict()` on `X_test` to get the forecasts.
3.  **Inverse Transform:** We convert the normalized predictions and actual values back to their original price scale for interpretation.
4.  **Calculate Metrics:** We compute a comprehensive suite of evaluation metrics, including RMSE, MAE, MAPE, and R-squared.
5.  **Baseline Comparison:** We compare the LSTM model's performance against a simple "Naive Forecast" baseline (predicting today's price is the same as yesterday's). This helps quantify the value added by our complex model.

In [ ]:
if not data.empty:
    # --- 1. Load the best model saved by ModelCheckpoint
    # The 'restore_best_weights=True' in EarlyStopping might make this redundant,
    # but loading explicitly is a robust practice.
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print("✅ Best model loaded from file for evaluation.")
    except Exception as e:
        print(f"⚠️ Could not load model from {MODEL_PATH}. Using the model from the last training epoch. Error: {e}")

    # --- 2. Make Predictions ---
    predicted_prices_scaled = model.predict(X_test)

    # --- 3. Inverse Transform ---
    # Important: Use the 'target_scaler' that was fitted only on the 'Close' price
    predicted_prices = target_scaler.inverse_transform(predicted_prices_scaled)
    actual_prices = target_scaler.inverse_transform(y_test.reshape(-1, 1))

    # --- 4. Calculate Performance Metrics ---
    def calculate_metrics(y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
        r2 = r2_score(y_true, y_pred)
        return rmse, mae, mape, r2

    lstm_rmse, lstm_mae, lstm_mape, lstm_r2 = calculate_metrics(actual_prices, predicted_prices)

### <a id="subsection8_1"></a>
### 8.1 Baseline Model Comparison

In [ ]:
if not data.empty:
    # --- Naive Forecast Baseline ---
    # The naive forecast predicts the price of day 't' to be the price of day 't-1'.
    # We get the 'Close' price from the original dataframe that corresponds to the test set.
    test_data_start_index = split_index + WINDOW_SIZE
    # Naive prediction for the first test day is the close price of the day before it.
    naive_predictions = data['Close'].iloc[test_data_start_index - 1 : -1].values

    # Ensure the lengths match for comparison
    actuals_for_naive = actual_prices[:len(naive_predictions)]

    naive_rmse, naive_mae, naive_mape, naive_r2 = calculate_metrics(actuals_for_naive, naive_predictions)

In [ ]:
if not data.empty:
    # --- Display Metrics in a Comparison Table ---
    metrics_data = {
        'Metric': ['RMSE (USD)', 'MAE (USD)', 'MAPE (%)', 'R-squared (R²)'],
        'LSTM Model': [f"{lstm_rmse:.4f}", f"{lstm_mae:.4f}", f"{lstm_mape:.4f}", f"{lstm_r2:.4f}"],
        'Naive Baseline': [f"{naive_rmse:.4f}", f"{naive_mae:.4f}", f"{naive_mape:.4f}", f"{naive_r2:.4f}"]
    }
    metrics_df = pd.DataFrame(metrics_data)

    print("\n--- Model Performance Comparison ---")
    display(metrics_df.style.hide_index())

### <a id="subsection8_2"></a>
### 8.2 Visualizing Results

In [ ]:
if not data.empty:
    # --- Visualize Predictions vs Actual Prices ---
    # Get the dates that correspond to the test set
    test_dates = data.index[split_index + WINDOW_SIZE:]

    fig_results = go.Figure()
    fig_results.add_trace(go.Scatter(x=test_dates, y=actual_prices.flatten(),
                                     mode='lines', name='Actual Prices',
                                     line=dict(color='cyan')))
    fig_results.add_trace(go.Scatter(x=test_dates, y=predicted_prices.flatten(),
                                     mode='lines', name='LSTM Predicted Prices',
                                     line=dict(color='magenta', dash='dash')))

    fig_results.update_layout(
        title=f'LSTM Prediction vs. Actual Prices for {STOCK_TICKER}',
        xaxis_title='Date',
        yaxis_title='Stock Price (USD)',
        template='plotly_dark',
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
    )
    fig_results.show()

### <a id="subsection8_3"></a>
### 8.3 Exporting Predictions

In [ ]:
if not data.empty:
    # --- Create a DataFrame with predictions and offer for download ---
    predictions_df = pd.DataFrame({
        'Date': test_dates,
        'Actual_Price': actual_prices.flatten(),
        'Predicted_Price': predicted_prices.flatten()
    })
    
    # Save to CSV
    csv_filename = f'{STOCK_TICKER}_predictions.csv'
    predictions_df.to_csv(csv_filename, index=False)

    print(f"\n✅ Predictions saved to '{csv_filename}'.")
    # Offer file for download if in Colab
    if IS_COLAB:
        try:
          files.download(csv_filename)
          print(f"Download started for {csv_filename}.")
        except Exception as e:
          print(f"Could not automatically download file. Please find it in the Colab file explorer.")

## <a id="section9"></a>
## 9. Saving and Loading the Model & Scalers

### What's happening here?
We have already saved the model and scalers during the training phase. This section demonstrates how to load them back from disk for future use, which is essential for making predictions without retraining.

1.  **Loading:** We load the saved model (`.h5` file) and the scaler objects (`.joblib` files).
2.  **Verification:** We use the loaded model to make a prediction and compare it to the original model's prediction to ensure they are identical.

In [ ]:
if not data.empty and os.path.exists(MODEL_PATH):
    print(f" artifacts have been saved in: {SAVE_DIR}")

    # --- Load the Saved Model and Scalers ---
    try:
        loaded_model = tf.keras.models.load_model(MODEL_PATH)
        loaded_feature_scaler = joblib.load(FEATURE_SCALER_PATH)
        loaded_target_scaler = joblib.load(TARGET_SCALER_PATH)
        print("\n✅ Successfully loaded the saved model and scalers from disk.")
        
        # --- Verification ---
        original_prediction = model.predict(X_test[:1], verbose=0)
        loaded_prediction = loaded_model.predict(X_test[:1], verbose=0)
    
        if np.allclose(original_prediction, loaded_prediction):
            print("✅ Verification successful: Loaded model prediction matches original model.")
        else:
            print("❌ Verification failed: Predictions do not match.")

    except Exception as e:
        print(f"❌ Error loading artifacts: {e}")
        loaded_model = None
else:
    print("Skipping model loading as model artifacts were not found.")
    loaded_model = None

## <a id="section10"></a>
## 10. Live Prediction on New Data

### What's happening here?
This function simulates a real-world prediction for the next trading day. It fetches the latest available data, processes it using the *same pipeline and saved scalers* from training, and feeds it into the loaded model to generate a forecast.

In [ ]:
def predict_next_day(ticker, model_path, feature_scaler_path, target_scaler_path):
    """
    Loads a trained model and scalers to predict the next day's closing price.
    
    Args:
        ticker (str): The stock ticker symbol.
        model_path (str): Path to the saved Keras model file.
        feature_scaler_path (str): Path to the saved feature scaler joblib file.
        target_scaler_path (str): Path to the saved target scaler joblib file.
    """
    print(f"\n--- Generating Live Prediction for {ticker} ---")
    try:
        # 1. Load model and scalers
        if not all(os.path.exists(p) for p in [model_path, feature_scaler_path, target_scaler_path]):
            print("❌ Missing model or scaler file. Please train the model first.")
            return None
        
        model_to_use = tf.keras.models.load_model(model_path)
        feature_scaler_to_use = joblib.load(feature_scaler_path)
        target_scaler_to_use = joblib.load(target_scaler_path)

        # 2. Fetch latest data (enough to calculate indicators and form a window)
        # Fetch more than needed to ensure indicators can be calculated
        latest_data = yf.download(ticker, period='6mo', progress=False)
        if len(latest_data) < WINDOW_SIZE:
            print("❌ Not enough recent data to form a prediction window.")
            return None

        # 3. Calculate necessary features
        latest_data['OBV'] = calculate_obv(latest_data) # Uses the function defined earlier
        latest_data.dropna(inplace=True)

        # 4. Get the last WINDOW_SIZE days of the required features
        last_window_data = latest_data[FEATURES_TO_USE].tail(WINDOW_SIZE)

        # 5. Scale the data using the *loaded* scaler from training
        scaled_window = feature_scaler_to_use.transform(last_window_data)

        # 6. Reshape for the model [1, window_size, num_features]
        X_pred = np.reshape(scaled_window, (1, WINDOW_SIZE, len(FEATURES_TO_USE)))

        # 7. Predict
        predicted_scaled_price = model_to_use.predict(X_pred, verbose=0)

        # 8. Inverse transform to get the actual price value
        predicted_price = target_scaler_to_use.inverse_transform(predicted_scaled_price)

        return predicted_price[0][0]

    except Exception as e:
        print(f"❌ An error occurred during live prediction: {e}")
        return None

In [ ]:
# --- Execute the live prediction ---
if not data.empty:
    next_day_prediction = predict_next_day(STOCK_TICKER, MODEL_PATH, FEATURE_SCALER_PATH, TARGET_SCALER_PATH)
    if next_day_prediction is not None:
        last_close = data['Close'].iloc[-1]
        print(f"\nLast available closing price ({data.index[-1].strftime('%Y-%m-%d')}): ${last_close:.2f}")
        print(f"📈 Predicted closing price for the next trading day for {STOCK_TICKER}: ${next_day_prediction:.2f}")

## <a id="section11"></a>
## 11. Advanced Topics & Future Work

This section explores advanced concepts and future development directions.

### Sentiment Analysis Integration
Research has shown that integrating sentiment scores from financial news can improve model accuracy. A full implementation would require:
1.  **Data Acquisition:** Using a news API (e.g., NewsAPI, Bloomberg, Reuters) to fetch real-time text data.
2.  **Temporal Alignment:** Calculating and aggregating sentiment scores (using a tool like VADER or a transformer model) for each trading day.
3.  **Integration:** Adding the normalized sentiment score as an additional feature to the model's input `X`.

### API Deployment for Real-time Inference
For production use, the prediction function should be deployed as a RESTful API. This would allow trading platforms and dashboards to request predictions programmatically. A simple implementation could use a web framework like Flask or FastAPI.

### Robustness: Fallback Data Sources
For a mission-critical system, relying on a single data source is risky. A robust implementation would include fallback logic.

```python
def get_data_robust(ticker):
    try:
        # Try primary source
        data = yf.download(ticker)
        if data.empty: raise ValueError("No data from yfinance")
        return data
    except Exception as e:
        print(f"Primary source failed: {e}. Trying fallback...")
        # Try secondary source (e.g., Alpha Vantage, IEX Cloud)
        # data = alpha_vantage.get_daily(...)
        return data # or None if all fail
```

### Memory Management for Large Datasets
While stock data is typically manageable, for extremely large datasets (e.g., tick-level data over decades), memory can be an issue.
- **Data Types:** Use memory-efficient data types in pandas (e.g., `float32` instead of `float64`).
- **Garbage Collection:** We use `del` and `gc.collect()` to manually free up memory after variables are no longer needed.
- **Generators/tf.data:** For datasets that don't fit in memory, use Python generators or the `tf.data` API to feed data to the model in batches without loading everything at once.

## 🎉 Notebook Complete!

You have successfully implemented a professional-grade LSTM model for stock price prediction. The model architecture and parameters provide a strong foundation for financial time-series forecasting.

### Key Achievements:
- ✅ Built a stacked LSTM model with customizable parameters
- ✅ Implemented a robust data preprocessing and feature engineering pipeline
- ✅ Established a naive baseline for meaningful model validation
- ✅ Implemented model and scaler saving/loading for reliable, stateless predictions
- ✅ Created live prediction functionality for real-time inference

### Next Steps:
1. **Experiment with different stocks** by changing the `STOCK_TICKER` parameter.
2. **Tune hyperparameters** or **add more features** (e.g., sentiment scores) to potentially improve accuracy.
3. **Deploy the model** as an API for production use.
4. **Explore ensemble methods**, such as combining LSTM with other models like GRU or ARIMA.

**Disclaimer:** Stock market prediction is inherently uncertain. This model is an educational tool and should be used to inform decisions, not as a guarantee of future performance. Always perform your own research and due diligence.